*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
import numpy as np
# import os
import pandas as pd
# import shutil
import tensorflow as tf
# import urllib.request

In [ ]:
# Predict the message type.  Returns list containing probability and label.
# prediction = predict_message(msg)
# print(prediction)
# >>> [0.008318834938108921, 'ham']
def predict_message(model, msg):
    prediction = model.predict(np.array([msg])).flatten()[0]
    print('msg:  {} type:  {}'.format(msg, prediction))

    if prediction > 0:
        return [prediction, 'ham']
    else:
        return [prediction, 'spam']

In [ ]:
def test_predictions(model):
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. "
        "just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(model, msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge.  Great job!")
    else:
        print("You haven't passed yet.  Keep trying.")

In [ ]:
# Pandas dataframe to tensorflow.data dataset.
# Requires whole dataframe, with features to use and target.
# https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers
def df_to_ds(df, target, shuffle=True, batch=32):
    features = df.copy()
    labels = features.pop(target)

    ds = tf.data.Dataset.from_tensor_slices(
        (features['message'].values, labels.values))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(features))

    ds = ds.batch(batch)
    ds = ds.prefetch(batch)

    return ds

In [ ]:
HAM = 1
SPAM = 0

# get data files
!wget https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv
!wget https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv

train_file = "train-data.tsv"
val_file = "valid-data.tsv"

train_d = pd.read_csv(train_file, sep='\t', names=['label', 'message'])
val_d = pd.read_csv(val_file, sep='\t', names=['label', 'message'])

train_d['target'] = np.where(train_d['label'] == 'ham', HAM, SPAM)
train_d = train_d.drop(columns=['label'])
val_d['target'] = np.where(val_d['label'] == 'ham', HAM, SPAM)
val_d = val_d.drop(columns=['label'])

TRAIN_MSGS = train_d.shape[0]
VAL_MSGS = val_d.shape[0]

# Features.
train_f = train_d.copy()
val_f = val_d.copy()

# Labels (targets).
train_l = train_f.pop('target')
val_l = val_f.pop('target')

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_ds = df_to_ds(train_d, 'target', shuffle=True, batch=BATCH_SIZE)
val_ds = df_to_ds(val_d, 'target', shuffle=False, batch=BATCH_SIZE)

VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_ds.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
)

val_loss, val_acc = model.evaluate(val_ds)

print('Validation Loss: {}'.format(val_loss))
print('Validation Accuracy: {}'.format(val_acc))

In [ ]:
test_predictions(model)